출처: https://docs.qiime2.org/2019.7/tutorials/fmt/

# Fecal microbiota transplant (FMT) study: an exercise

This document is intended to be run after the moving pictures tutorial. It is designed to introduce a few new ideas, and to be an exercise in applying the tools that were explored in that document.

The data used in this tutorial is derived from a Fecal Microbiome Transplant study

This tutorial dataset is a subsample of the data generated for this study.

These data were sequenced on two Illumina MiSeq sequencing runs.

# Obtain data files



In [1]:
!wget \
  -O "sample-metadata.tsv" \
  "https://data.qiime2.org/2019.7/tutorials/fmt/sample_metadata.tsv"

--2019-09-05 10:35:29--  https://data.qiime2.org/2019.7/tutorials/fmt/sample_metadata.tsv
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://docs.google.com/spreadsheets/d/1D3-Z20qK_jJoONXqHbVYffI5kyf6YinQ1F0f-2dnDbU/export?gid=0&format=tsv [following]
--2019-09-05 10:35:30--  https://docs.google.com/spreadsheets/d/1D3-Z20qK_jJoONXqHbVYffI5kyf6YinQ1F0f-2dnDbU/export?gid=0&format=tsv
Resolving docs.google.com (docs.google.com)... 216.58.220.206, 2404:6800:4005:80d::200e
Connecting to docs.google.com (docs.google.com)|216.58.220.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tab-separated-values]
Saving to: `sample-metadata.tsv'

    [ <=>                                   ] 11,526      --.-K/s   in 0.01s   

2019-09-05 10:35:31 (1.08 MB/s) - `sample-metadata.tsv' saved [11526]



Next, download the demultiplexed sequences that we’ll use in this analysis.

In this tutorial we’ll work with a small subsample of the complete sequence data so that the commands will run quickly.

In [2]:
!wget \
  -O "fmt-tutorial-demux-1.qza" \
  "https://data.qiime2.org/2019.7/tutorials/fmt/fmt-tutorial-demux-1-1p.qza"

--2019-09-05 10:35:57--  https://data.qiime2.org/2019.7/tutorials/fmt/fmt-tutorial-demux-1-1p.qza
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2019.7/tutorials/fmt/fmt-tutorial-demux-1-1p.qza [following]
--2019-09-05 10:35:58--  https://s3-us-west-2.amazonaws.com/qiime2-data/2019.7/tutorials/fmt/fmt-tutorial-demux-1-1p.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 54.231.168.160
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|54.231.168.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2165499 (2.1M) [binary/octet-stream]
Saving to: `fmt-tutorial-demux-1.qza'

100%[======================================>] 2,165,499   1.59M/s   in 1.3s    

2019-09-05 10:36:00 (1.59 MB/s) - `fmt-tutorial-demux-1.qza' saved [21654

In [3]:
!wget \
  -O "fmt-tutorial-demux-2.qza" \
  "https://data.qiime2.org/2019.7/tutorials/fmt/fmt-tutorial-demux-2-1p.qza"

--2019-09-05 10:36:09--  https://data.qiime2.org/2019.7/tutorials/fmt/fmt-tutorial-demux-2-1p.qza
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2019.7/tutorials/fmt/fmt-tutorial-demux-2-1p.qza [following]
--2019-09-05 10:36:10--  https://s3-us-west-2.amazonaws.com/qiime2-data/2019.7/tutorials/fmt/fmt-tutorial-demux-2-1p.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.237.64
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.237.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 935392 (913K) [binary/octet-stream]
Saving to: `fmt-tutorial-demux-2.qza'

100%[======================================>] 935,392     1.13M/s   in 0.8s    

2019-09-05 10:36:11 (1.13 MB/s) - `fmt-tutorial-demux-2.qza' saved [935392/9

# Sequence quality control

We’ll begin by performing quality control on the demultiplexed sequences using DADA2, but this time we’ll run the denoise-single command on each set of demultiplexed sequences individually.

In [4]:
!qiime demux summarize \
  --i-data fmt-tutorial-demux-2.qza \
  --o-visualization demux-summary-2.qzv

Saved Visualization to: demux-summary-2.qzv


In [5]:
from qiime2 import Visualization
Visualization.load('demux-summary-2.qzv')

<visualization: Visualization uuid: 0936081a-b5a4-484a-9285-efccaeea585b>

Here the quality seems relatively low in the first few bases, and then seems to stay relatively high through the end of the reads. We’ll therefore trim the first 13 bases from each sequence and truncate the sequences at 150 bases.

In [6]:
!qiime dada2 denoise-single \
  --p-trim-left 13 \
  --p-trunc-len 150 \
  --i-demultiplexed-seqs fmt-tutorial-demux-2.qza \
  --o-representative-sequences rep-seqs-2.qza \
  --o-table table-2.qza \
  --o-denoising-stats stats-2.qza

Saved FeatureTable[Frequency] to: table-2.qza
Saved FeatureData[Sequence] to: rep-seqs-2.qza
Saved SampleData[DADA2Stats] to: stats-2.qza


# Viewing denoising stats

The denoise-single commands return basic statistics about the denoising process, and can be visualized with the following:

In [7]:
!qiime metadata tabulate \
  --m-input-file stats-2.qza \
  --o-visualization denoising-stats-2.qzv

Saved Visualization to: denoising-stats-2.qzv


In [8]:
Visualization.load('denoising-stats-2.qzv')

<visualization: Visualization uuid: 7fd9f6c2-680e-40cd-b175-cbb536592328>

# Merging denoised data

The denoise-single command is the last step in this analysis that needs to be run on a per sequencing run basis.

In [10]:
!qiime feature-table summarize \
  --i-table table-2.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample-metadata.tsv

Saved Visualization to: table.qzv


In [11]:
Visualization.load('table.qzv')

<visualization: Visualization uuid: 7ab19b2b-3c0d-4574-8b87-c0313d12fcd1>

In [12]:
!qiime feature-table tabulate-seqs \
  --i-data rep-seqs-2.qza \
  --o-visualization rep-seqs.qzv

Saved Visualization to: rep-seqs.qzv


In [13]:
Visualization.load('rep-seqs.qzv')

<visualization: Visualization uuid: d7a2ba38-9279-4e20-b026-919709d47112>

# Diversity analysis

Now that you have FeatureTable[Frequency] and FeatureData[Sequence] objects, you’re ready to begin exploring the composition of these samples in the context of their metadata. 